In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Button, VBox, Output
from IPython.display import display

from computer_vision.src.constants import FolderPath

# Global variable to store the current figure for screenshot saving.
last_fig = None


def sample_function(x):
    """
    Define a piecewise constant function with clearly distinct regions.
    The function f(x) takes on:
      - 0.5 when x < -10,
      - 1.5 when -10 <= x < 0,
      - 0.2 when 0 <= x < 10,
      - 1.0 when x >= 10.
    """
    return np.piecewise(
        x,
        [x < -10, (x >= -10) & (x < 0), (x >= 0) & (x < 10), x >= 10],
        [0.5, 1.5, 0.2, 1.0],
    )


def visualize_box_convolution(a=5.0, s=0.0):
    global last_fig
    # Set up the spatial domain.
    N = 1024  # Number of sample points
    L = 50.0  # Domain will be [-L, L]
    x = np.linspace(-L, L, N)
    dx = x[1] - x[0]

    # Evaluate the sample function on the grid.
    f_val = sample_function(x)

    # --- Define the Shifted Box Filter ---
    # The shifted box filter is centered at s with width a.
    # It equals 1 where |x - s| <= a/2, and 0 elsewhere.
    shifted_box = np.where(np.abs(x - s) <= a / 2, 1.0, 0.0)

    # Compute the local integral of f_val over the window defined by the shifted box.
    local_integral = np.trapezoid(f_val * shifted_box, x)

    # --- Compute the Full Convolution ---
    # Define a symmetric box (centered at 0) for the convolution.
    symmetric_box = np.where(np.abs(x) <= a / 2, 1.0, 0.0)
    # Compute the convolution (using 'same' to keep the result size identical to x).
    conv = np.convolve(f_val, symmetric_box, mode="same") * dx
    # Find the index corresponding to the shift s.
    idx = np.argmin(np.abs(x - s))
    conv_value = conv[idx]

    # --- Plotting ---
    fig, axs = plt.subplots(2, 1, figsize=(10, 12))

    # Top subplot: sample function with the shifted box filter overlay.
    axs[0].plot(x, f_val, label="f(x) (Sample Function)", lw=2)
    axs[0].plot(x, shifted_box, label="Shifted Box Filter", lw=2, color="orange")
    axs[0].fill_between(
        x,
        f_val,
        where=(np.abs(x - s) <= a / 2),
        color="green",
        alpha=0.3,
        label="Integration Region",
    )
    axs[0].set_title(
        f"f(x) with Shifted Box Filter (center = {s:.2f}, width = {a:.2f})\nLocal Integral = {local_integral:.3f}"
    )
    axs[0].set_xlabel("x")
    axs[0].set_ylabel("Amplitude")
    axs[0].legend()

    # Bottom subplot: full convolution result.
    axs[1].plot(x, conv, label="Convolution f * box", lw=2, color="C2")
    axs[1].plot(
        x[idx], conv_value, "ro", label=f"Value at x = {s:.2f}\n= {conv_value:.3f}"
    )
    axs[1].set_title("Convolution of f(x) with the Box Filter (Moving Integral)")
    axs[1].set_xlabel("x")
    axs[1].set_ylabel("Local Integral")
    axs[1].legend()

    plt.tight_layout()
    plt.show()

    # Update the global variable with the current figure.
    last_fig = fig


# Create interactive sliders for the box width 'a' and the shift 's'
interactive_plot = interact(
    visualize_box_convolution, a=(1.0, 20.0, 1.0), s=(-20.0, 20.0, 0.5)
)

# --- Create and Set Up the Screenshot Button ---
save_button = Button(description="Save Screenshot")
output = Output()


def save_screenshot(button):
    with output:
        if last_fig is not None:
            # Create a filename based on the current time.
            filename = (
                "convolution_box_visualisation_"
                + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                + ".png"
            )
            last_fig.savefig(FolderPath.Images.joinpath(filename))
            print(f"Screenshot saved to {filename}")
        else:
            print("No figure available to save yet.")


save_button.on_click(save_screenshot)

# Display the button and output widget together.
display(VBox([save_button, output]))

interactive(children=(FloatSlider(value=5.0, description='a', max=20.0, min=1.0, step=1.0), FloatSlider(value=…